In [5]:

#prototyp dataframe generator: 
#soll relevante matadaten des Datensatzes in einem dataframe zusammenbringen und es ermöglichen darauf NLP methoden anzuwenden 

#NEXT STEPS:
    #- all xml files last 2 years pipen
    #- Data cleaning on fields,
    #- Sentiment Analysis with Textblob on comments, speeches for subjects-> (Sentiment(polarity=0.7, subjectivity=0.6000000000000001))
    #- time series analysis on same topics


In [6]:
#import der libaries
import pandas as pd
from bs4 import BeautifulSoup
import requests
import lxml     #benötigt für xml format

#plenarprotokoll laden #hier: über request
req = requests.get('https://www.bundestag.de/resource/blob/898114/e7ace72bfd61d683b1bd36c50ec0813e/20042-data.xml')
soup = BeautifulSoup(req.content, 'xml')

#with open('./DIS25NLP/venv/plenarprotokoll.xml') as f:
#   soup = BeautifulSoup(f, 'xml')

In [7]:
#dataframe format
df = pd.DataFrame(columns=['wahlperiode', 'sitzungsnr', 'datum', 'tagespunkt', 'thema','redner','kommentare', 'rede'])

In [8]:
kopfdaten = soup.find_all('kopfdaten')
tagesordnungspunkte = soup.find_all('tagesordnungspunkt')

# tagespunkte loopen
for tpunkt in tagesordnungspunkte:
    
    #look if speech here
    try:
        #gets tagespunkt
        tagespunkt = (tpunkt.attrs)['top-id']
        #print(tagespunkt)

        if  tpunkt.find('rede'):
            reden_soup = tpunkt.find_all('rede')
            # print('here are speeches')

            # show topic(s) of the tagespunkt:
            try:    
                
                topic_soup = tpunkt.find_all(klasse="T_fett")
                topics = [] 
                for topic in topic_soup: 
                    t = topic.text
                    topics.append(t)
            except:
                pass


            # reden loopen kommentare sowie reden und redner infos speichern           
            for rede in reden_soup:
                
                #comments in speech
                comments_list = []
                kommentare_soup = rede.find_all('kommentar')
                for i in kommentare_soup:
                    comments_list.append(i.text)
                
                reden_list = []
                
                reden_soup = rede.find_all('p')
                
                for text in reden_soup:
                    if text.find('redner'):
                        continue
                    reden_list.append(text.text)
                
                try:
                    fraktion = rede.fraktion.text
                    
                except AttributeError:
                    pass
                try:
                    rolle = rede.rolle.text
                except AttributeError:
                    pass


                #row entrag für dataframe modelieren
                row_dict  = {
                        'wahlperiode': kopfdaten[0].wahlperiode.text,
                        'sitzungsnr':kopfdaten[0].sitzungsnr.text,
                        'datum':kopfdaten[0].datum.attrs['date'],
                        'tagespunkt': tagespunkt, 
                        'thema': topics, 
                        'redner': [rede.vorname.text , rede.nachname.text,],
                        'kommentare': comments_list,
                        'rede': reden_list,
            
                        }
                
                # 
                if fraktion:
                    row_dict['redner'].append(fraktion)
                elif rolle:
                    row_dict['redner'].append(rolle)
                else:
                    row_dict['redner'].append('fraktionslos')


                df = df.append(row_dict,ignore_index=True)
                


        else:
            pass
            #print('no speeches found')

          

    except:
        #print('no speeches in this tagesordnungpunkt')
        pass

df.to_csv('./dataframeProto.csv')




df


,wahlperiode,sitzungsnr,datum,tagespunkt,thema,redner,kommentare,rede
0,20,42,03.06.2022,Einzelplan 11,"[Drucksache 20/1916, Ausweitung der Minijobs k...","[Silke, Launert, CDU/CSU]",[(Markus Kurth [BÜNDNIS 90/DIE GRÜNEN]: Die Ei...,[Sehr geehrte Frau Präsidentin! Liebe Kollegin...
1,20,42,03.06.2022,Einzelplan 11,"[Drucksache 20/1916, Ausweitung der Minijobs k...","[Hubertus, Heil, CDU/CSU]",[(Beifall bei der SPD und dem BÜNDNIS 90/DIE G...,[Frau Präsidentin! Meine sehr geehrten Damen u...
2,20,42,03.06.2022,Einzelplan 11,"[Drucksache 20/1916, Ausweitung der Minijobs k...","[Ulrike, Schielke-Ziesing, AfD]","[(Beifall bei der AfD), (Dirk Heidenblut [SPD]...",[Sehr geehrte Frau Präsidentin! Sehr geehrter ...
3,20,42,03.06.2022,Einzelplan 11,"[Drucksache 20/1916, Ausweitung der Minijobs k...","[Andreas, Audretsch, BÜNDNIS 90/DIE GRÜNEN]",[(Beifall beim BÜNDNIS 90/DIE GRÜNEN und bei d...,[Frau Präsidentin! Sehr geehrte Damen und Herr...
4,20,42,03.06.2022,Einzelplan 11,"[Drucksache 20/1916, Ausweitung der Minijobs k...","[Gesine, Lötzsch, DIE LINKE]","[(Beifall bei der LINKEN), (Beifall bei der LI...",[Herzlichen Dank. – Frau Präsidentin! Meine se...
...,...,...,...,...,...,...,...,...
63,20,42,03.06.2022,Tagesordnungspunkt VI,"[Drucksache 20/2074, Rentner beim Entlastungsp...","[Anja, Schulz, FDP]",[(Beifall bei der FDP – Matthias W. Birkwald [...,[Sehr geehrte Frau Präsidentin! Liebe Kollegin...
64,20,42,03.06.2022,Tagesordnungspunkt VI,"[Drucksache 20/2074, Rentner beim Entlastungsp...","[Matthias W., Birkwald, DIE LINKE]","[(Beifall bei der LINKEN), (Beifall bei Abgeor...",[Sehr geehrte Frau Präsidentin! Meine Damen un...
65,20,42,03.06.2022,Tagesordnungspunkt VI,"[Drucksache 20/2074, Rentner beim Entlastungsp...","[Michael, Gerdes, SPD]",[(Beifall bei der SPD sowie bei Abgeordneten d...,[Frau Präsidentin! Liebe Kolleginnen und Kolle...
66,20,42,03.06.2022,Tagesordnungspunkt VI,"[Drucksache 20/2074, Rentner beim Entlastungsp...","[Kai, Whittaker, CDU/CSU]","[(Beifall bei der CDU/CSU), (Beifall bei der C...",[Frau Präsidentin! Werte Kolleginnen und Kolle...


In [9]:

! pip install -U textblob
! python -m textblob.download_corpora

     |████████████████████████████████| 636 kB 2.4 MB/s            
     |████████████████████████████████| 1.5 MB 2.8 MB/s            
     |████████████████████████████████| 306 kB 3.0 MB/s            
     |████████████████████████████████| 749 kB 3.0 MB/s            
     |████████████████████████████████| 97 kB 2.7 MB/s            
     |████████████████████████████████| 78 kB 2.9 MB/s            
[nltk_data] Downloading package brown to /home/poseidon/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package punkt to /home/poseidon/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /home/poseidon/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/poseidon/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package conll2000 to
[nltk_data]     /home/poseidon/nltk_

In [11]:
from textblob import TextBlob


In [17]:
pol = lambda x: TextBlob(x).sentiment.polarity
sub = lambda x: TextBlob(x).sentiment.subjectivity



type(df['rede'])
#df['polarity'] =  (df['rede']).apply(pol)
#df


pandas.core.series.Series

In [19]:
string = "The sentiment property returns a namedtuple of the form Sentiment(polarity, subjectivity). The polarity score is a float within the range [-1.0, 1.0]. The subjectivity is a float within the range [0.0, 1.0] where 0.0 is very objective and 1.0 is very subjective."

test = TextBlob(string)
test.sentiment


Sentiment(polarity=0.1, subjectivity=0.215)